In [777]:
from bs4 import BeautifulSoup
import configparser
import requests
import pandas as pd
import pickle
from constants_funcs import get_session, int_or_float_or_str
import json
import numpy as np
import time
from dateutil.parser import parse
import csv
with open('constants.json', 'r') as d:
    constants = json.load(d)

In [182]:
sess = get_session()
sess

using existing login session


In [820]:
oneday_question_cols = ['url', 'qnum', 'question', 'answer', 'pct_correct', 'pct_money', 'pct_pay', 'pct_a', 'pct_b', 'pct_c', 'pct_d', 'pct_e', 'pct_r']
oneday_answer_dtypes = {
    'Season': 'string', 'PlayerName': 'string', 'Rank': int, 'Percentile': int, 'Tot': int, 'q1c': int, 'q2c': int, 'q3c': int,
       'q4c': int, 'q5c': int, 'q6c': int, 'q7c': int, 'q8c': int, 'q9c': int, 'q10c': int, 'q11c': int, 'q12c': int, 'q1v': int,
       'q2v': int, 'q3v': int, 'q4v': int, 'q5v': int, 'q6v': int, 'q7v': int, 'q8v': int, 'q9v': int, 'q10v': int, 'q11v': int,
       'q12v': int
}
question_exceptions = {
    '/oneday.php?unreliablenarrators_10': "",
    '/oneday.php?unreliablenarrators_11': "This fictional commander's short gregarious and dubious exploits of grandeur delighted young audiences on such cartoons as Tennessee Tuxedo and his Tales, Hippity Hop, and The Rocky and Bullwinkle Show. The segment usually involved the commander standing next to a globe while an exasperated listener endured a remarkable tale.",
    '/oneday.php?4283_11': "You're in the Top 3 and have made it to the Final Tribal Council! Now the power shifts to the jury, nine people that you had a hand in voting out will decide who they think is most worthy of the title of Sole Survivor and the million-dollar check that goes along with it.  Hopefully you have managed the jury well and voted them out in such a way that they respect your game and are not bitter. Upon elimination, jury members are sent to a resort where they physically recover and discuss who they might vote for to win the game at Final Tribal Council. What is the name of this resort? It shares a name (and is probably named after) a fictional ranch from a TV show that ran for 14 seasons.  This name is also redacted from this audio clip featuring MC Chris Noble from Survivor: Ghost Island. Transcription of audio quote: 'Living the dream on [REDACTED]  Swung for the fences I call it Sammy Sosa.  Juice all day and toss porks in the roaster.  Probst on deck feel the burn when he hosts ya.'",
    '/oneday.php?3507_3': "The second Mrs. de Winter nervously agrees to host Manderley's annual costume ball, and is persuaded to dress - unwittingly - in the same costume worn by her predecessor, Rebecca, at the final party before her death. What malevolent housekeeper orchestrates this mischief?",
    '/oneday.php?3507_4': "Jennie Livingston's documentary 'Paris is Burning' shone a light on the LGBTQ+ drag ball culture of New York in the 1980s. Renowned drag queen Willi Ninja, featured in the film, became known as the godfather/grandfather of what heavily-stylized form of dance?",
    '/oneday.php?3507_6': "Whose ball is this? He had more than one, and all proceeds went to Amnesty International.",
    '/oneday.php?murderballads_0': "",
    '/oneday.php?murderballads_1': "",
    '/oneday.php?murderballads_2': "",
    '/oneday.php?murderballads_3': "",
    '/oneday.php?murderballads_4': "",
    '/oneday.php?murderballads_5': "",
    '/oneday.php?murderballads_6': "",
    '/oneday.php?murderballads_7': "",
    '/oneday.php?murderballads_8': "",
    '/oneday.php?murderballads_9': "",
    '/oneday.php?murderballads_10': "",
    '/oneday.php?murderballads_11': "",
    '/oneday.php?supportingtitlecharacters_11': "The director of what blockbuster has insisted that its title ultimately refers not to the main character, but to Detective John Blake, as stressed by the film's final scene? The director is quoted: is a symbol. He can be anybody, and that was very important to us.",
    '/oneday.php?interactivefiction_9': "Out of Infocom's 35 published works of IF, only one was written by a woman. Styled after a romance novel and set in the late 17th century, this is also the only Infocom game where the player's character is clearly and specifically defined by the story as female. Name either the game or its author.",
    '/oneday.php?asongoficeandfire_7': "The bulk of the continent of Westeros, conquered by Aegon I and his forces, is referred to as 'the Seven Kingdoms,' a realm comprised of kingdoms such as The North, The Iron Islands, and Dorne. Name any one of the remaining four kingdoms.",
    '/oneday.php?conspiracytheories_5': "OBAMA LEFT- INTERNATIONALIST GRAFT ACORN STYLE ORGANIZATIONS REVOLUTIONARIES HIDDEN AGENDAS Whose televised chalkboard displayed this failed acrostic in 2009?",
    '/oneday.php?conspiracytheories_6': "Larry Shippers are theorists (the majority of them teenagers) who believe which two individuals are secretly in love?",
    '/oneday.php?conspiracytheories_7': "This collage depicts and interprets a number of pieces of artwork on display where?",
    '/oneday.php?philosophy_8': "This fictional character is expounding a central idea from an influential American work of political philosophy published in 1971. (Parenthetically, and not for nothing, he always talks like this, even when he's not mansplaining to a roomful of hotties.) Name either the book or its author.",
    '/oneday.php?philosophy_9': "'It is the greatest happiness of the greatest number that is the measure of right and wrong.' This principle, described by Jeremy Bentham in 1776 as a 'fundamental axiom,' is central to what philosophy?",
    '/oneday.php?philosophy_10': "Name the philosopher who, according to a familiar song lyric, 'was a boozy beggar who could think you under the table.'",
    '/oneday.php?philosophy_11': "In a famous passage from Leviathan, Thomas Hobbes paints a stark portrait of human existence in a state of nature, with 'no navigation...no commodious building...no arts, no letters, no society,' and concludes with a string of five baleful adjectives describing 'the life of man' under such conditions. The first two, often omitted when this line is quoted, are 'solitary' and 'poor'; what are the other three?",
    '/oneday.php?justimagesstamps_5': "What film is this?",
    '/oneday.php?justimagesstamps_6': "This 1999 stamp, only the second US stamp to commemorate an American artist, is faithfully based upon an iconic 1949 Life magazine cover photo, but for two significant alterations. The stamp designer changed the image from black-and-white to color, and was also directed to remove what object from the original?",
    '/oneday.php?justimagesstamps_7': "A phrase stamped on this envelope (redacted here) makes it a particularly notable collectible. What phrase is it?",
    '/oneday.php?justimagesstamps_8': "This stamp was issued by what country?",
    '/oneday.php?justimagesstamps_9': "This stamp series features what particular group of women?",
    '/oneday.php?justimagesstamps_10': "This philatelic first was issued in what year?",
    '/oneday.php?justimagesstamps_11': "Pittsburgh native Burt Todd helped establish the postage stamp system of this country and was responsible for many of its unique stamp issues, including the stamp pictured here. The audio on the stamp features Todd reciting various facts about the country. What country is it?",
    '/oneday.php?etymology_7': "What in the world of entertainment got its name from this song?",
    '/oneday.php?etymology_8': "This word comes from the sport of falconry, and originally meant a bird trapped as an adult and very difficult to train, unlike a bird captured when a nestling. One can well imagine the exhausted and frustrated look on the trainer after trying to train such a bird, and probably that led to the modern usage of the word. So what's the good word?",
    '/oneday.php?etymology_9': "Which sobriquet originated from this sentence in a mid-19th century report in The Times, London (edited and key word blanked out)? 'When...silence and darkness have settled down upon those miles of prostrate sick, she may be observed alone, with a little ____ in her hand, making her solitary rounds.'",
    '/oneday.php?etymology_10': "Which company, founded by a former teacher of Transcendental Meditation, is named after a cross-legged sitting posture originating in meditative practices of ancient India?",
    '/oneday.php?etymology_11': "Which Hollywood movie from the 1950s took its title from that of psychologist Robert Lindner's 1944 classic book, which had the subtitle 'The Hypnoanalysis of a Criminal Psychopath', and records the successful analysis and hypnosis of a criminal psychopath, Harold (it should be noted that the film told an unrelated story)?",
    '/oneday.php?alfredhitchcock_9': "Who was responsible for the creative design of this scene featured in the 1945 film Spellbound?",
    '/oneday.php?alfredhitchcock_10': "After embezzling $40,000 from her employer, Phoenix officeworker Marion Crane meets her death the following evening where?",
    '/oneday.php?alfredhitchcock_11': "What 1942 film features a scene in which the villain falls to his death from the torch atop the Statue of Liberty?"
}
answer_exceptions = {
    '/oneday.php?unreliablenarrators_10': "",
    '/oneday.php?unreliablenarrators_11': "COMMANDER MCBRAGG",
    '/oneday.php?murderballads_0': "",
    '/oneday.php?murderballads_1': "",
    '/oneday.php?murderballads_2': "",
    '/oneday.php?murderballads_3': "",
    '/oneday.php?murderballads_4': "",
    '/oneday.php?murderballads_5': "",
    '/oneday.php?murderballads_6': "",
    '/oneday.php?murderballads_7': "",
    '/oneday.php?murderballads_8': "",
    '/oneday.php?murderballads_9': "",
    '/oneday.php?murderballads_10': "",
    '/oneday.php?murderballads_11': "",
    '/oneday.php?philosophy_8': "A THEORY OF JUSTICE, JOHN RAWLS",
    '/oneday.php?philosophy_9': "UTILITARIANISM",
    '/oneday.php?philosophy_10': "MARTIN HEIDEGGER",
    '/oneday.php?philosophy_11': "NASTY, BRUTISH, SHORT",
    '/oneday.php?justimagesstamps_5': "BREWSTER'S MILLIONS",
    '/oneday.php?justimagesstamps_6': "A CIGARETTE",
    '/oneday.php?justimagesstamps_7': "RETURN TO SENDER",
    '/oneday.php?justimagesstamps_8': "IRAN",
    '/oneday.php?justimagesstamps_9': "JAMES BOND GIRLS",
    '/oneday.php?justimagesstamps_10': "1948",
    '/oneday.php?justimagesstamps_11': "BHUTAN",
    '/oneday.php?etymology_7': "WHO WANTS TO BE A MILLIONAIRE?",
    '/oneday.php?etymology_8': "HAGGARD",
    '/oneday.php?etymology_9': "LADY WITH THE LAMP",
    '/oneday.php?etymology_10': "LOTUS SOFTWARE",
    '/oneday.php?etymology_11': "REBEL WITHOUT A CAUSE",
    '/oneday.php?alfredhitchcock_9': "SALVADOR DALI",
    '/oneday.php?alfredhitchcock_10': "BATES MOTEL",
    '/oneday.php?alfredhitchcock_11': "SABOTEUR"
}

In [807]:
def init_oneday_metadata(): # only needed to run once!
    oneday_pd = pd.DataFrame(columns={
        'url': pd.Series(dtype='str'),
        'is_classic': pd.Series(dtype='bool'),
        'title': pd.Series(dtype='str'),
        'date': pd.Series(dtype='str'),
        'num_players': pd.Series(dtype='int'),
        'smiths': pd.Series(dtype='object'),
        'league_champ': pd.Series(dtype='str'),
        'top_league_pts': pd.Series(dtype='int')
    })
    oneday_pd.to_csv('./oneday_metadata.csv', index=False)
def check_for_new_1ds(existing1DSData: pd.DataFrame) -> tuple[list[str], pd.DataFrame]:
    soup = BeautifulSoup(
        sess.get(constants['ONEDAYS']).text, 'html.parser')
    midseason_classics, one_days = soup.find_all('table', {'summary': 'Data table of One-Day results'})
    new_metadata = []
    # list of [url, is_classic, title, date, num_players, smiths(regular), league_champ(classic), top_league_pts(classic)]
    existing_urls = existing1DSData['url']
    for tr in midseason_classics.find('tbody').find_all('tr'):
        mid_url = tr.find('a')['href']
        if mid_url not in existing_urls: # if mid_url not in existing 1ds data
            title, date, num_players, league_champ, top_league_pts = [td.text.strip() for td in tr.find_all('td')]
            new_metadata.append([mid_url, True, title, str(parse(date).date()), int(num_players), None, league_champ, int(top_league_pts)])
    for tr in one_days.find('tbody').find_all('tr'):
        if tr.find('a'):
            mid_url = tr.find('a')['href']
            if mid_url not in existing_urls: # if mid_url not in existing 1ds data
                title, date, num_players = [td.text.strip() for td in tr.find_all('td')[:3]]
                smiths = [int(a['href'][14:]) for a in tr.find_all('td')[3].find_all('a') if a['href'] != '/profiles.php?']
                new_metadata.append([mid_url, False, title, str(parse(date).date()), num_players, smiths, None, None])
    new_df = pd.concat(
        [existing1DSData, pd.DataFrame(new_metadata, columns=existing1DSData.columns)], ignore_index=True, axis=0
    ).astype({
        'url': str,
        'is_classic': bool,
        'title': str,
        'date': str,
        'num_players': int,
        'smiths': object,
        'league_champ': str,
        'top_league_pts': pd.Int64Dtype()
    })
    return [meta[0] for meta in new_metadata], new_df
def get_oneday_questions(oneday_url: str, existing_1ds_metadata: pd.DataFrame) -> pd.DataFrame:
    soup = BeautifulSoup(
        sess.get(constants['LLHEADER'] + oneday_url).text, 'html.parser')
    qs, ans, q_stats = [], [], []
    if soup.find('div', attrs={'id': 'qs_close'}): # new html
        qs_div = soup.find('div', attrs={'id': 'qs_close'})
        qs = [question_exceptions[f'{oneday_url}_{idx}'] if f'{oneday_url}_{idx}' in question_exceptions.keys() else '.'.join(''.join([
            piece if type(piece) == str else piece.text for piece in p.contents[:p.contents.index('\n')]
        ]).split('.')[1:]).strip() for idx, p in enumerate(
            qs_div.find_all('p')[:12] + ['']*(12-len(qs_div.find_all('p')[:12])))]
        ans = ['']*12 if oneday_url == '/oneday.php?murderballads' else [ # more shitty formatting
            answer_exceptions[f'{oneday_url}_{idx}'] if f'{oneday_url}_{idx}' in answer_exceptions.keys()else ans_div.find(
            'div', attrs={'class': 'a-red'}).text.strip() for idx, ans_div in enumerate(
                qs_div.find_all('div',attrs={'class': 'indivqAnswerwrapper'}) +
                ['']*(12-len(qs_div.find_all('div', attrs={'class': 'indivqAnswerwrapper'})[:12])))]
    elif soup.find('div', attrs={'class': 'contentbox_new'}):
        qs_div = soup.find('div', attrs={'class': 'contentbox_new'})
        qs = ['.'.join(
            ''.join(
                [c if type(c) == str else c.text for c in q.contents[
                    :[c.name for c in q.contents].index('b')
                    ]
                ]
            ).split('.')[1:]).strip() for q in qs_div.find_all('p')]
        ans = [a.text.strip() for a in qs_div.find_all('b') if not (
            (oneday_url == '/oneday/number50.shtml' and a.text.strip() == 'more'))]
    elif soup.find('td', attrs={'class': 'BxQA'}):
        qs_div = soup.find('td', attrs={'class': 'BxQA'})
        # qs_div.find_all('p')
        qs = ['.'.join(
            ''.join(
                [c if type(c) == str else c.text for c in q.contents[
                    :[c.name for c in q.contents].index('b')
                    ]
                ]
            ).split('.')[1:]).strip() for q in qs_div.find_all('p')[:12]]
        ans = [a.text.strip() for a in qs_div.find_all('b') if not (
            a.text.strip() == "TODAY'S QUESTIONSAND ANSWERS" or
            a.text.strip() == "TODAY'S QUESTIONS AND ANSWERS" or
            (oneday_url == '/oneday/colbertreport.shtml' and a.text.strip() == 'STRONG') or
            (oneday_url == '/oneday/73.shtml' and a.text.strip() in ['out', 'man', 'reach']) or 
            (oneday_url == '/oneday/24.shtml' and a.text.strip() == 'H5N1'))]
    else:
        print('unfamiliar with this HTML layout.')
        return qs, ans, q_stats
    new_data_cutoff = parse('2013-05-11').date()
    if parse(existing_1ds_metadata[existing_1ds_metadata['url'] == oneday_url]['date'].values[0]).date() >= new_data_cutoff:
        newsoup = BeautifulSoup(
            sess.get(constants['ONEDAYS'] + '/results.php?' + oneday_url.split('?')[-1] + '&12').text, 'html.parser'
        )
        q_stats = [[None]]*12 if oneday_url == '/oneday.php?machines' else [ #more shitty formatting
            [int(td.text) for td in tr.find_all('td')[1:]] for tr in newsoup.find('table', attrs={'class': 'tbl_q'}).find_all('tr')[1:13]]
    elif oneday_url in ['/oneday/1.shtml', '/oneday/2.shtml']:
        q_stats = [[100-int(td.text)] for td in soup.find_all('td', attrs={'class': 'BxAPVal'})[:12]]
    else:
        q_stats = [[100-int(td.text if str(td.text).isnumeric() else '0')] for td in soup.find('tr', attrs={'bgcolor': '#ffffff'}).find_all('td')[1:13]]
    try:
        assert len(qs) == 12, f'Should have 12 questions, returned {len(qs)}'
        assert len(ans) == 12, f'Should have 12 answers, returned {len(ans)}'
        assert len(q_stats) == 12, f'Should have 12 question stats, returned {len(q_stats)}'
    except:
        raise Exception(f'failed assertion on {oneday_url}: {len(qs)}, {len(ans)}, {len(q_stats)}')
    new_pd = []
    for idx in range(12):
        row_to_insert = [oneday_url, idx+1, qs[idx], ans[idx], *q_stats[idx]]
        row_to_insert.extend([None]*(len(oneday_question_cols) - len(q_stats[idx]) - 4))
        new_pd.append(row_to_insert)
    new_pd = pd.DataFrame(new_pd, columns=oneday_question_cols).astype({
        'url': 'string', 'qnum': int, 'question': 'string', 'answer': 'string', 'pct_correct': pd.Int64Dtype(),
        'pct_money': pd.Int64Dtype(), 'pct_pay': pd.Int64Dtype(), 'pct_a': pd.Int64Dtype(), 'pct_b': pd.Int64Dtype(),
        'pct_c': pd.Int64Dtype(), 'pct_d': pd.Int64Dtype(), 'pct_e': pd.Int64Dtype(), 'pct_r': pd.Int64Dtype()
    })
    return new_pd
def get_oneday_player_data(oneday_url: str) -> pd.DataFrame:
    if oneday_url.endswith('.shtml'):
        print(f'{oneday_url} does not have an associated CSV')
        return None
    csv_url = constants['ONEDAYS'] + '/csv/' + oneday_url.split('?')[-1] + '.csv'
    try:
        ret = sess.get(csv_url)
        if ret.status_code == 404:
            print(f'{oneday_url} does not have an associated CSV')
            return None
        cr = list(csv.reader(ret.content.decode('latin-1').splitlines(), delimiter=','))
        cr[0][3] = 'Percentile'
        df = pd.DataFrame(cr[1:], columns=cr[0]).astype(oneday_answer_dtypes)
        return df
    except Exception as e:
        print(f'{oneday_url} does not have an associated CSV')
        print(e)
        return None

In [714]:
oneday_metadata_pd = pd.read_csv('./oneday_metadata.csv')
new_urls, oneday_metadata_pd = check_for_new_1ds(oneday_metadata_pd)
oneday_question_pd = pd.DataFrame(columns=oneday_question_cols)
oneday_pd_list = []
dfs = []
failed_urls = []
for idx, url in enumerate(new_urls):
    if idx % 500 == 0: print(f'completed {idx}')
    try: dfs.append(get_oneday_questions(url, oneday_metadata_pd))
    except Exception as e:
        failed_urls.append(url)
        print(f'failed on {url}: {e}')

In [763]:
for idx, url in enumerate(failed_urls):
    failed_urls.remove(url)
    if idx % 500 == 0: print(f'completed {idx}')
    try: dfs.append(get_oneday_questions(url, oneday_metadata_pd))
    except Exception as e:
        failed_urls.append(url)
        print(f'failed on {url}: {e}')

completed 0


In [769]:
mega_df = pd.concat(dfs, ignore_index=True)
oneday_metadata_pd.to_csv('./oneday_metadata.csv', index=False)
mega_df.to_csv('./oneday_questions.csv', index=False)
mega_df.shape, mega_df.dtypes

((33060, 13),
 url            string
 qnum            int64
 question       string
 answer         string
 pct_correct     Int64
 pct_money       Int64
 pct_pay         Int64
 pct_a           Int64
 pct_b           Int64
 pct_c           Int64
 pct_d           Int64
 pct_e           Int64
 pct_r           Int64
 dtype: object)

In [809]:
player_dfs = []
failed_urls = []
for idx, url in enumerate(new_urls):
    if idx % 500 == 0: print(f'completed {idx}')
    try:
        df = get_oneday_player_data(url)
        if df is not None: player_dfs.append(df)
    except Exception as e:
        failed_urls.append(url)
        print(f'failed on {url}: {e}')
mega_player_df = pd.concat(player_dfs, ignore_index=True).astype(oneday_answer_dtypes)
mega_player_df.to_csv('./oneday_player_history.csv', index=False)
mega_player_df.shape, mega_player_df.dtypes

completed 0
/oneday.php?classic_74 does not have an associated CSV
/oneday.php?classic_73 does not have an associated CSV
/oneday.php?classic_72 does not have an associated CSV
/oneday.php?classic_71 does not have an associated CSV
/oneday.php?classic_70 does not have an associated CSV
/oneday.php?classic_69 does not have an associated CSV
/oneday.php?classic_68 does not have an associated CSV
/oneday.php?classic_67 does not have an associated CSV
/oneday.php?classic_66 does not have an associated CSV
completed 500
completed 1000
/oneday.php?trentreznor does not have an associated CSV
/oneday.php?luckyjohnsons does not have an associated CSV
/oneday.php?nerdypuns does not have an associated CSV
/oneday.php?earthquakes2 does not have an associated CSV
/oneday.php?coenbrothers does not have an associated CSV
/oneday.php?justimagesinaccuratemaps does not have an associated CSV
/oneday.php?nerdyjokes2 does not have an associated CSV
/oneday.php?goats does not have an associated CSV
complet